In [32]:
import pandas as pd

# List of multilingual text and label files
text_files = [r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\arabic\train_text.txt", 
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\spanish\train_text.txt",
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\all\train_text.txt", 
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\german\train_text.txt",
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\english\train_text.txt",
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\french\train_text.txt",
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\portuguese\train_text.txt",
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\hindi\train_text.txt", 
              r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\italian\train_text.txt"]

label_files = [r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\arabic\train_labels.txt",
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\spanish\train_labels.txt",
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\all\train_labels.txt", 
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\german\train_labels.txt", 
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\english\train_labels.txt",
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\french\train_labels.txt",
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\portuguese\train_labels.txt",
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\hindi\train_labels.txt",
               r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\data\sentiment\italian\train_labels.txt"]

# Initialize empty lists
all_texts, all_labels = [], []

# Loop through files and merge
for text_file, label_file in zip(text_files, label_files):
    with open(text_file, "r", encoding="utf-8") as f_text, open(label_file, "r", encoding="utf-8") as f_label:
        texts = f_text.readlines()
        labels = f_label.readlines()  # Keep labels as strings

        all_texts.extend(texts)
        all_labels.extend(labels)

# Save text and labels separately
with open("train_text_combined.txt", "w", encoding="utf-8") as f_text:
    f_text.writelines(all_texts)

with open("train_labels_combined.txt", "w", encoding="utf-8") as f_label:
    f_label.writelines(all_labels)

print("✅ Merged all multilingual datasets into train_text_combined.txt & train_labels_combined.txt!")

✅ Merged all multilingual datasets into train_text_combined.txt & train_labels_combined.txt!


In [33]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from langdetect import detect
from textblob import TextBlob

In [34]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pbsns\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
from nltk.corpus import stopwords

# Define stopwords for Hindi & Arabic manually
hindi_stopwords = {"का", "की", "के", "है", "था", "थी", "हो", "कर", "से", "पर", "यह", "कि", "और"}
arabic_stopwords = {"في", "من", "على", "عن", "إلى", "هذا", "ذلك", "لكن", "كان", "ما", "مع", "كما", "لأن"}

# Load multilingual stopwords
stopwords_dict = {
    "en": set(stopwords.words("english")),
    "fr": set(stopwords.words("french")),
    "de": set(stopwords.words("german")),
    "es": set(stopwords.words("spanish")),
    "it": set(stopwords.words("italian")),
    "pt": set(stopwords.words("portuguese")),
    "hi": hindi_stopwords,  # Use manually defined set
    "ar": arabic_stopwords  # Use manually defined set
}

# Sample slang dictionary (expand as needed)
slang_dict = {
    "u": "you",
    "r": "are",
    "btw": "by the way",
    "smh": "shaking my head",
    "omg": "oh my god",
    "idk": "I don't know",
    "thx": "thanks",
    "plz": "please"
}

print("Stopwords loaded successfully!")


Stopwords loaded successfully!


In [36]:
# Function to clean text
def clean_text(text):
    """Removes URLs, mentions, and special characters."""
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions (@user)
    text = re.sub(r"[^a-zA-Z0-9\s\u0600-\u06FF\u0900-\u097F]", "", text)  # Keep only letters & numbers
    return text.lower()

In [37]:
# Function to handle slang & spelling corrections
def correct_text(text):
    """Replaces slang and corrects misspellings."""
    words = text.split()
    corrected_words = [slang_dict.get(word, word) for word in words]
    return " ".join(corrected_words)


In [38]:
# Function to remove stopwords based on detected language
def remove_stopwords(text):
    """Removes stopwords for detected language."""
    try:
        lang = detect(text)
        words = text.split()
        filtered_words = [word for word in words if word not in stopwords_dict.get(lang, set())]
        return " ".join(filtered_words)
    except:
        return text  # Return original if language detection fails

In [39]:
# Function to preprocess text
def preprocess_text(text):
    """Applies all preprocessing steps."""
    text = clean_text(text)
    text = correct_text(text)
    text = remove_stopwords(text)
    return text


In [40]:
# Load dataset
def load_dataset(text_path, label_path):
    """Loads text and labels from files."""
    with open(text_path, "r", encoding="utf-8") as f:
        texts = f.readlines()
    with open(label_path, "r", encoding="utf-8") as f:
        labels = [int(label.strip()) for label in f.readlines()]
    return pd.DataFrame({"text": texts, "label": labels})



In [45]:

train_data = load_dataset(r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\notebooks\train_text_combined.txt",r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\notebooks\train_labels_combined.txt")
train_data["cleaned_text"] = train_data["text"].apply(preprocess_text)

In [46]:
# Save cleaned data
train_data.to_csv("cleaned_train.csv", index=False)
print("Preprocessing completed! Saved to cleaned_train.csv")

Preprocessing completed! Saved to cleaned_train.csv
